# 20 Newsgroups

In [3]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn.decomposition import TruncatedSVD
from sklearn.datasets import fetch_20newsgroups_vectorized
from daal.algorithms.svd import Batch, data, singularValues, rightSingularMatrix, leftSingularMatrix
from daal.data_management import HomogenNumericTable, BlockDescriptor_Float64, readOnly

import time

ModuleNotFoundError: No module named 'daal'

In [ ]:
# Данная функция переводит из NumericTable в numpy array

def getArrayFromNT(table, nrows=0):

    bd = BlockDescriptor_Float64()

    if nrows == 0:
        nrows = table.getNumberOfRows()

    table.getBlockOfRows(0, nrows, readOnly, bd)
    npa = bd.getArray()
    table.releaseBlockOfRows(bd)

    return npa

# Вывод NumericTable в консоль

def printNT(table, nrows = 0, message=''):
    npa = getArrayFromNT(table, nrows)
    print(message, "\n", npa)

## Решение в scikit-learn

In [ ]:
newsgroups = fetch_20newsgroups_vectorized(subset='train', remove = ('headers', 'footers', 'quotes'))

# Размерность данных
print("Размерность данных \n", newsgroups.data, "\n")
# Применяем SVD к данным
svd = TruncatedSVD(n_components = 3000, algorithm = "randomized")

t = time.process_time()
svd.fit(newsgroups.data)
t = time.process_time() - t

# Доля объясненной вариации
print(" Доля объясненной вариации \n", svd.explained_variance_ratio_.sum(), "\n")
# График роста доли объясненной вариации
var_nwsd = np.round(np.cumsum(svd.explained_variance_ratio_), decimals=4)
plt.plot(np.arange(1,3001), var_nwsd)
plt.ylabel('Variation')
plt.xlabel('Number of PC')

# Время выполнения
print("Время выполнения (секунд) \n", t, "\n")

Размерность данных 
 <class 'scipy.sparse._csr.csr_matrix'> 



## Решение в pyDAAL

In [ ]:
# Транспонируем матрицу данных и переводим из sparse в dense

nwsd = newsgroups.data
nwsd = nwsd.transpose()
nwsd_dense = nwsd.toarray()

print("Размерность данных \n", nwsd_dense.shape, "\n")
# Перевод в NumericTable
nwsd_dense_nt = HomogenNumericTable(nwsd_dense)
# Выполнение SVD
algorithm = Batch()
algorithm.input.set(data, nwsd_dense_nt)

t = time.process_time()
result = algorithm.compute()
t = time.process_time() - t

# Доля объясненной вариации
ev = np.square(getArrayFromNT(result.get(singularValues)))
var_all = ev.sum()
var = ev/var_all
var_explained = np.round(np.cumsum(var), decimals=4)

print(" Доля объясненной вариации \n", var_explained[2999].sum(), "\n")

plt.plot(np.arange(1,3000), var_explained[0:2999])
plt.ylabel('Variation')
plt.xlabel('Number of PC')

# Время выполнения
print("Время выполнения (секунд) \n", t, "\n")